The UEFA Champions League, often called the Champions League, is a preeminent annual soccer competition that captivates fans worldwide. Established in 1955 as the European Champion Clubs' Cup, it evolved into the UEFA Champions League in 1992, broadening its appeal. The modern format features 32 top-tier club teams selected based on their domestic league performance, adding to the intrigue.


![Stadium.jpg](Stadium.jpg)


This electrifying event transcends sports, becoming a celebration of unity, culture, and national pride. Fans, draped in their countries' colors, create an electric atmosphere, making the tournament as much about the spectators as the players.Financially, the Champions League is a lifeline for clubs, boosting revenues and offering transformative opportunities. Nevertheless, it sparks debates about wealth disparities in European soccer.



![Duel_football](Duel_football.jpg)

The Champions League is synonymous with historic rivalries, underdog triumphs, and individual brilliance. For players, it represents a career pinnacle, while for fans, it's a cultural phenomenon. The iconic anthem and rituals enrich the soccer experience. In 200 words, the UEFA Champions League is the epitome of European soccer excellence, offering unforgettable moments, financial rewards, and a unique cultural impact, with 32 top clubs adding to its allure.






## Schema name: `SOCCER`
## Table Name(s): `TBL_UEFA_2020` | `TBL_UEFA_2021` | `TBL_UEFA_2022`
### Note : All three tables have same column names and data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` | 
Note that *in Snowflake all databases, tables, and columns are **upper case*** by default.

You will execute SQL queries to answer three questions, as listed in the instructions.

In [8]:
import pandas as pd
import pandasql
import duckdb

In [18]:
uefa_2020 = pd.read_csv('uefa_2020.csv')

In [22]:
# TEAM_HOME_WITH_MOST_GOALS

result = duckdb.query("""
SELECT u0."TEAM_NAME_HOME", 
       u0."TEAM_HOME_SCORE"
FROM uefa_2020 u0
ORDER BY u0."TEAM_HOME_SCORE" DESC 
LIMIT 3;
""").to_df()

result

,TEAM_NAME_HOME,TEAM_HOME_SCORE
0,PSG,5
1,Manchester United,5
2,Barcelona,5


In [24]:
uefa_2021 = pd.read_csv('uefa_2021.csv')

In [26]:
# TEAM_WITH_MAJORITY_POSSESSION
import duckdb

result = duckdb.query("""
SELECT
    CASE 
        WHEN u1."POSSESSION_HOME" > u1."POSSESSION_AWAY" THEN u1."TEAM_NAME_HOME"
        WHEN u1."POSSESSION_AWAY" > u1."POSSESSION_HOME" THEN u1."TEAM_NAME_AWAY"
        ELSE NULL 
    END AS TEAM_NAME,
    COUNT(*) AS GAME_COUNT
FROM uefa_2021 u1
GROUP BY TEAM_NAME
ORDER BY GAME_COUNT DESC
LIMIT 1;
""").to_df()

result

,TEAM_NAME,GAME_COUNT
0,Liverpool,9


In [32]:
import chardet
import pandas as pd

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

file_path = 'uefa_2022.csv'
encoding = detect_encoding(file_path)

print(f"Detected encoding: {encoding}")

try:
    uefa_2022 = pd.read_csv(file_path, encoding=encoding)
    print("File read successfully with detected encoding.")
    print(uefa_2022.head())
except UnicodeDecodeError as e:
    print(f"Error reading file with detected encoding: {e}")
    print("Try a different encoding manually.")
except Exception as e:
    print(f"An error occurred: {e}")

Detected encoding: ISO-8859-1
File read successfully with detected encoding.
                       STAGE      DATE   PENS  PENS_HOME_SCORE  \
0   Group stage: Matchday 1   6-9-2022  False            False   
1   Group stage: Matchday 1   6-9-2022  False            False   
2   Group stage: Matchday 1   6-9-2022  False            False   
3   Group stage: Matchday 1   6-9-2022  False            False   
4   Group stage: Matchday 1   6-9-2022  False            False   

   PENS_AWAY_SCORE     TEAM_NAME_HOME    TEAM_NAME_AWAY  TEAM_HOME_SCORE  \
0            False      Dinamo Zagreb           Chelsea                1   
1            False           Salzburg          AC Milan                1   
2            False             Celtic       Real Madrid                0   
3            False         RB Leipzig  Shakhtar Donetsk                1   
4            False  Borussia Dortmund         København                3   

   TEAM_AWAY_SCORE POSSESSION_HOME  ... TOTAL_SHOTS_HOME  TOTAL_SHOTS

In [34]:
# TEAM_WON_DUEL_LOST_GAME_STAGE_WISE
import duckdb

result = duckdb.query("""
WITH duel_lost AS (
    SELECT 
        u2."STAGE",
        CASE
            WHEN u2."DUELS_WON_HOME" > u2."DUELS_WON_AWAY" 
                 AND u2."TEAM_HOME_SCORE" < u2."TEAM_AWAY_SCORE" 
                THEN u2."TEAM_NAME_HOME"
            WHEN u2."DUELS_WON_AWAY" > u2."DUELS_WON_HOME" 
                 AND u2."TEAM_AWAY_SCORE" < u2."TEAM_HOME_SCORE" 
                THEN u2."TEAM_NAME_AWAY"
            ELSE NULL
        END AS TEAM_LOST    
    FROM uefa_2022 u2
)
SELECT 
    "STAGE",
    TEAM_LOST
FROM duel_lost
WHERE TEAM_LOST IS NOT NULL;
""").to_df()

result

,STAGE,TEAM_LOST
0,Group stage: Matchday 1,Chelsea
1,Group stage: Matchday 1,København
2,Group stage: Matchday 1,Juventus
3,Group stage: Matchday 1,Maccabi Haifa
4,Group stage: Matchday 1,Rangers
5,Group stage: Matchday 1,Liverpool
6,Group stage: Matchday 1,Porto
7,Group stage: Matchday 1,Bayer Leverkusen
8,Group stage: Matchday 1,Viktoria Plze?
9,Group stage: Matchday 1,Marseille
